In [ ]:
# Turn off Tensorflow warnings

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Imports

import numpy as np
import tensorflow as tf
import keras as k
import pickle
from sklearn.preprocessing import StandardScaler
import glob
import random

tf.keras.utils.disable_interactive_logging()

## Helper functions

def load_object(filename):
    with open(filename, 'rb') as imp:  # Overwrites any existing file.
        res = pickle.load(imp)
    return res

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

# Functions to manipulate traces

def add_new_burst(trace, index, sizes, interval):
    l = len(sizes)
    s = sum(sizes)
    new_row = np.array([l, s, 100.0*l, 800.0*s, interval, s/l])
    res = trace.insert(trace, index, new_row, axis=0)
    return np.stack(res)

def add_to_burst(trace, index, sizes, interval):
    old_row = trace[index]
    old_l, old_s = old_row[0], old_row[1]
    old_i = old_row[4]
    new_l = old_l + len(sizes)
    new_s = old_s + sum(sizes)
    new_i = (old_i*(old_l-1) + len(sizes)*interval)/(old_l+len(sizes)-1)
    new_row =  np.array([new_l, new_s, 100*new_l, 800*new_s, new_i, new_s/new_l])
    trace[index] = new_row
    return trace

# Default function for the purposes of this project

def add_to_bursts(trace, b_j):
    new_trace = []
    for index in range(trace.shape[1]):
        old_row = trace[0][index]
        old_l, old_s = old_row[0], old_row[1]
        old_i = old_row[4]
        new_l = old_l + b_j[index]
        new_s = old_s*(new_l/old_l)
        new_i = (old_i*(old_l-1) + b_j[index]*old_i)/(new_l-1) if new_l != 1 else 0
        new_r = new_s/new_l if new_l != 0 else 0
        new_row = tf.convert_to_tensor([new_l, new_s, 100*new_l, 800*new_s, new_i, new_s/new_l], dtype=tf.float32)
        new_trace.append(new_row)
    return tf.reshape(tf.stack(new_trace), (1, 5, 6))

In [ ]:
def mockingbird(source_trace, model, targets, **kwargs):
    number_of_time_series = source_trace.shape[0]
    res = []
    for idx in range(number_of_time_series):
        new_targets = [targ[idx].reshape(1, 5, 6) for targ in targets if targ.shape[0] > idx]
        new_source = source_trace[idx].reshape(1, 5, 6)
        res.append(_mockingbird(new_source, model, new_targets, **kwargs).reshape(5, 6))
    return np.array(res)


def _mockingbird(source_trace, model, targets, *,
                alpha=0.05, d=20, threshold_c=0.3,
                threshold_d=1, max_n=100,
                p=5):
    
    I_s = source_trace
    S = targets
    
    # Main procedure
    Y_s = predict(model, source_trace)[0]
    P_s = random.sample(S, p)
    I_t = argmin(P_s, lambda I: metric(I_s, I))
    I_s_new = np.copy(I_s)
    for i in range(max_n):
        gradient = compute_gradient(I_s_new, I_t).numpy()
        ind = np.where(gradient >= 0)
        I_s_new[0][ind] = np.dot((1 + alpha * gradient[ind]), I_s_new[0][ind])
        Y_s_new, P = predict(model, I_s_new, confidence=Y_s)
        if Y_s_new != Y_s and P < threshold_c:
            break
        
        if (i%d == 0 and Y_s_new == Y_s 
           and metric(I_s_new, I_s) < threshold_d):
            P_s = random.sample(S, p)
            I_t = argmin(P_s, lambda I: metric(I_s, I))
    return I_s_new

def argmin(S, f):
    minval = None; argm = None
    for s in S:
        if argm is None or f(s) < minval:
            minval = f(s)
            argm = s
    return argm

def predict(model, sample, confidence=None):
    scaled_sample = scaler.transform(sample.reshape(5, 6)).reshape(1, 5, 6)
    p = model.predict(scaled_sample)
    res = p.argmax(axis=1)
    if confidence != None:
        return res, p[0][confidence]
    return res

def compute_gradient(I1, I2):
    b_j = tf.constant([0.0, 0.0, 0.0, 0.0, 0.0], dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(b_j)
        dist = -metric(add_to_bursts(I1, b_j), I2)
    gradient = tape.gradient(dist, b_j)
    return gradient
    
def metric(a, b):
    return tf.norm(a-b)

def make_result_sane(inp):
    result = np.copy(inp)
    result[0] = np.ceil(inp[0])
    result[1] = np.ceil(inp[1])
    result[2] = 100*result[0]
    result[3] = 8*result[1]*100
    result[4] = inp[4] if result[0] > 1 else 0
    result[5] = result[1]/result[0] if result[0] != 0 else 0
    return result

In [ ]:
# Create dataset for Mockingbird algorithm

data_path = "Data/"
browse_data_path = data_path + "browse/"
chat_data_path = data_path + "chat/"
mail_data_path = data_path + "mail/"
p2p_data_path =  data_path + "p2p/"
scaler = load_object('scaler.pkl')

datasets = dict()
samples = dict()
sample_data = dict()
concatenated_sample_data = dict()
sample_labels = dict()


for (index, path) in  enumerate([browse_data_path, chat_data_path,
                      mail_data_path, p2p_data_path]):
    datasets[path] = list(glob.glob(path + "*.pcap"))
    samples[path] = random.sample(datasets[path],
                                  len(datasets[path])//10)
    sample_data[path] = []
    for sample_path in samples[path]:
        sample_data[path].append(np.loadtxt(sample_path, delimiter=',').reshape((-1,6)))
    concatenated_sample_data[path] = np.concatenate(sample_data[path])
    
total_data = np.concatenate([concatenated_sample_data[path] for path in sample_data])
scaler = StandardScaler().fit(total_data)
scaled_data = dict()
for data_type in sample_data:
    scaled_data[data_type] = scaler.transform(concatenated_sample_data[data_type])

for (index, path) in  enumerate([browse_data_path, chat_data_path,
                      mail_data_path, p2p_data_path]):
    if scaled_data[path].shape[0]%5 == 0:
        scaled_data[path] = np.array_split(scaled_data[path], scaled_data[path].shape[0]//5)
    else:
        scaled_data[path] = np.array_split(scaled_data[path][:-(scaled_data[path].shape[0]%5)], scaled_data[path].shape[0]//5)
        
    scaled_data[path] = np.stack(scaled_data[path])
    sample_labels[path] = np.tile(np.eye(4)[index], (scaled_data[path].shape[0], 1))

train_data = np.concatenate([scaled_data[path] for path in scaled_data])
train_labels = np.concatenate([sample_labels[path] for path in sample_labels])

save_object(sample_data, 'mockingbird_sample_data.pkl')
save_object(sample_labels, 'mockingbird_sample_labels.pkl')
save_object(train_data, 'mockingbird_train_data.pkl')
save_object(train_labels, 'mockingbird_train_labels.pkl')


In [ ]:

train_data = load_object('mockingbird_train_data.pkl')
train_labels = load_object('mockingbird_train_labels.pkl')

# Create LSTM detector for Mockingbird algorithm.

activation = k.activations.relu

mockingbird_detector_input = tf.keras.layers.Input(shape=(5, 6))
mockingbird_detector_e_1 = tf.keras.layers.LSTM(16, return_sequences=True)(mockingbird_detector_input)
mockingbird_detector_e_2 = tf.keras.layers.LSTM(136, return_sequences=False)(mockingbird_detector_e_1)
mockingbird_detector_d_repeat = tf.keras.layers.RepeatVector(5)(mockingbird_detector_e_2)
mockingbird_detector_d_1 = tf.keras.layers.LSTM(136, return_sequences=True)(mockingbird_detector_d_repeat)
mockingbird_detector_d_2 = tf.keras.layers.LSTM(16, return_sequences=True)(mockingbird_detector_d_1)
mockingbird_detector_output = k.layers.Dense(6, activation='sigmoid')(mockingbird_detector_d_2)
mockingbird_detector_sae_fcnn_1 = tf.keras.layers.Dense(64, activation=activation)(mockingbird_detector_e_2)
mockingbird_detector_sae_fcnn_2 = tf.keras.layers.Dense(24, activation=activation)(mockingbird_detector_sae_fcnn_1)
mockingbird_detector_sae_output = tf.keras.layers.Dense(4, activation='softmax')(mockingbird_detector_sae_fcnn_2)

mockingbird_detector_sae = tf.keras.Model(inputs=mockingbird_detector_input, outputs=mockingbird_detector_output, name='mockingbird_detector')
mockingbird_detector_sae.compile(loss='mse', optimizer='adam')
mockingbird_detector_sae_encoder = k.Model(mockingbird_detector_input, mockingbird_detector_e_2)

mockingbird_detector_sae_full = k.Model(inputs=mockingbird_detector_input, outputs=mockingbird_detector_sae_output, name='mockingbird_detector_full')
mockingbird_detector_sae_full.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Train 

mockingbird_detector_sae.fit(train_data, train_data, epochs=10, batch_size=16,
                             callbacks=[k.callbacks.ModelCheckpoint(filepath='mockingbird_detector.{epoch:02d}.keras'),
                             k.callbacks.EarlyStopping(monitor='loss')])

for layer in mockingbird_detector_sae.layers:
    layer.trainable = False

mockingbird_detector_sae_full.fit(train_data, train_labels, epochs=100, batch_size=16,
                                   callbacks=[k.callbacks.ModelCheckpoint(filepath='mockingbird_detector_full.{epoch:02d}.keras'),
                                   k.callbacks.EarlyStopping(monitor='loss')])


In [ ]:
from itertools import permutations

mockingbird_detector_sae_full = tf.keras.saving.load_model('mockingbird_detector_full.38.keras')
stacked_cnn_lstm_sae_full = tf.keras.saving.load_model('stacked_cnn_lstm_sae.53.keras')
scaler = load_object('scaler.pkl')

# Set up sources

data_path = "Data/"
browse_data_path = data_path + "browse/"
chat_data_path = data_path + "chat/"
mail_data_path = data_path + "mail/"
p2p_data_path =  data_path + "p2p/"

sample_data = load_object('mockingbird_sample_data.pkl')
sample_labels = load_object('mockingbird_sample_labels.pkl')
train_data = load_object('mockingbird_train_data.pkl')
train_labels = load_object('mockingbird_train_labels.pkl')

# Create sliced samples
def create_lstm_sample(sample):
    if sample.shape[0] % 5 == 0:
        s = sample
    else:
        s = sample[:-(sample.shape[0]%5)]
    return np.stack(np.array_split(s, sample.shape[0]//5), axis=0)

sliced_sample_data = dict()

for path in sample_data:
    sliced_sample_data[path] = []
    for sample in sample_data[path]:
        if sample.shape[0] >= 15:
            sliced = create_lstm_sample(sample)
            sliced_sample_data[path].append(sliced)

sources = dict()
for path in [browse_data_path, chat_data_path, p2p_data_path]:
    sources[path] = create_lstm_sample(random.choice([p for p in sample_data[path] if 50 >= p.shape[0] >= 15]))


# Set up targeted pools

targeted_pools = dict()
for path in [browse_data_path, chat_data_path,
            mail_data_path, p2p_data_path]:
    targeted_pools[path] = sliced_sample_data[path]


# Set up untargeted pools:

untargeted_pools = dict()
for path in [browse_data_path, chat_data_path,
            mail_data_path, p2p_data_path]:
    untargeted_pools[path] = [j for p in [browse_data_path, chat_data_path,mail_data_path, p2p_data_path]
        for j in targeted_pools[p]]

# Run algorithm for each pair and record the results in the logfile
    
with open('mockingbird.log', "a") as logfile:
    print("Untargeted:\n", file=logfile)
    logfile.flush()
    for path in [browse_data_path, chat_data_path, p2p_data_path]:
        print("Source folder:", path, file=logfile)
        source, pool = sources[path], untargeted_pools[path]
        print("Source trace:", source, file=logfile)
        logfile.flush()
        adv = mockingbird(source, mockingbird_detector_sae_full, pool)
        print("Adversarial raw:", adv, file=logfile)
        logfile.flush()
        sane_adv = np.apply_along_axis(make_result_sane, -1, adv)
        print("Adversarial sane:", sane_adv, file=logfile)
        logfile.flush()
        res = stacked_cnn_lstm_sae_full.predict(sane_adv).argmax(axis=-1)
        vals, counts = np.unique(res, return_counts=True)
        mr = vals[np.argmax(counts)]
        print("Predictions:", res, file=logfile)
        print("Majority Prediction:", mr, file=logfile)
        logfile.flush()
        
    print("Targeted:\n", file=logfile)
    for (path1, path2) in permutations([browse_data_path, chat_data_path, p2p_data_path], 2):
        print("Source folder:", path1, file=logfile)
        print("Target folder:", path2, file=logfile)
        logfile.flush()
        source, pool = sources[path1], targeted_pools[path2]
        print("Source trace:", source, file=logfile)
        logfile.flush()
        adv = mockingbird(source, mockingbird_detector_sae_full, pool)
        print("Adversarial raw:", adv, file=logfile)
        logfile.flush()
        sane_adv = np.apply_along_axis(make_result_sane, -1, adv)
        print("Adversarial sane:", sane_adv, file=logfile)
        logfile.flush()
        res = stacked_cnn_lstm_sae_full.predict(sane_adv).argmax(axis=-1)
        vals, counts = np.unique(res, return_counts=True)
        mr = vals[np.argmax(counts)]
        print("Predictions:", res, file=logfile)
        print("Majority Prediction:", mr, file=logfile)
        logfile.flush()
    print("END LOG", file=logfile)